In [1]:
# reflected location error in ozone data simulation

import torch
import torch.fft
import numpy as np
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis

import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time
from sklearn.neighbors import BallTree
# Custom imports


from GEMS_TCO import kernels_reparam_space_time_gpu as kernels_reparam_space_time_gpu

from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, BaseLogger

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle
from torch.nn import Parameter

# --- 1. CONFIGURATION ---
# Check for Mac GPU (MPS) first, then CUDA, then CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Optional: Force CPU if you encounter Cholesky errors later
# DEVICE = torch.device("cpu") 
DTYPE = torch.float32 if DEVICE.type == 'mps' else torch.float64

print(f"Simulating on: {DEVICE}")

# TRUE PARAMETERS
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map parameters
init_phi2 = 1.0 / init_range_lon
init_phi1 = init_sigmasq * init_phi2
init_phi3 = (init_range_lon / init_range_lat)**2
init_phi4 = (init_range_lon / init_range_time)**2

# Create Initial Parameters
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [
    torch.tensor([val], requires_grad=True, dtype=DTYPE, device=DEVICE)
    for val in initial_vals
]

# Mean Ozone
OZONE_MEAN = 260.0

# --- 2. EXACT COVARIANCE ---
def get_model_covariance_on_grid(lags_x, lags_y, lags_t, params):
    phi1, phi2, phi3, phi4 = torch.exp(params[0]), torch.exp(params[1]), torch.exp(params[2]), torch.exp(params[3])
    advec_lat, advec_lon = params[4], params[5]
    sigmasq = phi1 / phi2

    u_lat_eff = lags_x - advec_lat * lags_t
    u_lon_eff = lags_y - advec_lon * lags_t
    
    dist_sq = (u_lat_eff.pow(2) * phi3) + (u_lon_eff.pow(2)) + (lags_t.pow(2) * phi4)
    distance = torch.sqrt(dist_sq + 1e-8)
    
    return sigmasq * torch.exp(-distance * phi2)

# --- 3. FFT SIMULATION ---
def get_wrapped_covariance(lags_x, lags_y, lags_t, params, Lx_len, Ly_len, Lt_len):
    phi1, phi2, phi3, phi4 = torch.exp(params[0]), torch.exp(params[1]), torch.exp(params[2]), torch.exp(params[3])
    advec_lat, advec_lon = params[4], params[5]
    sigmasq = phi1 / phi2

    # [핵심] Folding Logic: 거리가 전체 길이의 절반을 넘어가면 반대편 거리로 계산
    # 예: 길이가 10인데 거리가 8이면, 사실상 반대편으로 2만큼 떨어진 것임.
    
    # 1. Advection 적용
    u_lat = lags_x - advec_lat * lags_t
    u_lon = lags_y - advec_lon * lags_t
    
    # 2. Wrap-around (Torus) 거리 계산
    # torch.remainder를 써서 주기성을 만듦
    u_lat_wrapped = torch.remainder(u_lat + Lx_len/2, Lx_len) - Lx_len/2
    u_lon_wrapped = torch.remainder(u_lon + Ly_len/2, Ly_len) - Ly_len/2
    t_wrapped     = torch.remainder(lags_t + Lt_len/2, Lt_len) - Lt_len/2

    dist_sq = (u_lat_wrapped.pow(2) * phi3) + (u_lon_wrapped.pow(2)) + (t_wrapped.pow(2) * phi4)
    distance = torch.sqrt(dist_sq + 1e-8)
    
    return sigmasq * torch.exp(-distance * phi2)

# --- 4. REGULAR GRID FUNCTIONS (FIXED WITH ROUNDING) ---

def make_target_grid(lat_start, lat_end, lat_step, lon_start, lon_end, lon_step, device, dtype):
    """
    Constructs a grid explicitly from start to end.
    CRITICAL: Includes rounding to 4 decimal places to prevent "Tensor size does not match" errors.
    """
    # 1. Generate Latitudes (Descending from 5.0)
    # We use a small epsilon to ensure the 'end' is included if it's a multiple
    lats = torch.arange(lat_start, lat_end - 0.0001, lat_step, device=device, dtype=dtype)
    lats = torch.round(lats * 10000) / 10000  # <--- FIX: Round to 4 decimals
    
    # 2. Generate Longitudes (Descending from 133.0)
    lons = torch.arange(lon_start, lon_end - 0.0001, lon_step, device=device, dtype=dtype)
    lons = torch.round(lons * 10000) / 10000  # <--- FIX: Round to 4 decimals

    print(f"Grid Generation debug: Lat Range {lats[0]:.4f}-{lats[-1]:.4f}, Lon Range {lons[0]:.4f}-{lons[-1]:.4f}")
    print(f"Unique Lats: {len(lats)}, Unique Lons: {len(lons)}")

    # 3. Meshgrid (indexing='ij' -> Lat is rows, Lon is cols)
    grid_lat, grid_lon = torch.meshgrid(lats, lons, indexing='ij')

    # 4. Flatten
    flat_lats = grid_lat.flatten()
    flat_lons = grid_lon.flatten()

    # 5. Stack
    center_points = torch.stack([flat_lats, flat_lons], dim=1)
    
    # Return grid AND dimensions (Nx, Ny) for verification
    return center_points, len(lats), len(lons)

def coarse_by_center_tensor(input_map_tensors: dict, target_grid_tensor: torch.Tensor):
    coarse_map = {}
    
    # BallTree requires CPU Numpy
    query_points_np = target_grid_tensor.cpu().numpy()
    query_points_rad = np.radians(query_points_np)
    
    for key, val_tensor in input_map_tensors.items():
        # Source locations (Perturbed)
        source_locs_np = val_tensor[:, :2].cpu().numpy()
        source_locs_rad = np.radians(source_locs_np)
        
        # NN Search
        tree = BallTree(source_locs_rad, metric='haversine')
        dist, ind = tree.query(query_points_rad, k=1)
        nearest_indices = ind.flatten()
        
        # Map values back to tensor
        indices_tensor = torch.tensor(nearest_indices, device=val_tensor.device, dtype=torch.long)
        gathered_vals = val_tensor[indices_tensor, 2]
        gathered_times = val_tensor[indices_tensor, 3]
        
        # Construct Regular Tensor
        new_tensor = torch.stack([
            target_grid_tensor[:, 0], # Regular Lat
            target_grid_tensor[:, 1], # Regular Lon
            gathered_vals,            # Mapped Value
            gathered_times            # Mapped Time
        ], dim=1)
        
        coarse_map[key] = new_tensor

    return coarse_map

# --- 3. FFT SIMULATION (FOLDING VERSION) ---

# [수정된 시뮬레이션 함수] 2배 뻥튀기 없이, Folding 공분산 사용
def generate_folded_gems_field(lat_coords, lon_coords, t_steps, params):
    Nx = len(lat_coords)
    Ny = len(lon_coords)
    Nt = t_steps
    
    # [차이점 1] 2배 확장(Padding) 하지 않음! 입력 크기 그대로 사용
    # 이미 1.25배 확장된 격자가 들어오므로 이걸로 충분함
    Px, Py, Pt = Nx, Ny, Nt
    
    print(f"Folded Simulation Grid: {Px} x {Py} x {Pt} (No 2x Padding)")
    
    dlat = float(lat_coords[1] - lat_coords[0])
    dlon = float(lon_coords[1] - lon_coords[0])
    dt = 1.0 
    
    # 전체 도메인 물리적 길이 (주기)
    Lx_len = abs(Px * dlat)
    Ly_len = abs(Py * dlon)
    Lt_len = abs(Pt * dt)
    
    # 격자 생성
    lags_x = torch.arange(Px, device=DEVICE, dtype=DTYPE) * dlat # dlat 부호 그대로
    lags_y = torch.arange(Py, device=DEVICE, dtype=DTYPE) * dlon
    lags_t = torch.arange(Pt, device=DEVICE, dtype=DTYPE) * dt
    
    # Meshgrid
    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')

    # [차이점 2] 일반 공분산 대신 'Wrapped(Folding)' 공분산 호출
    C_vals = get_wrapped_covariance(L_x, L_y, L_t, params, Lx_len, Ly_len, Lt_len)

    # FFT Simulation
    S = torch.fft.fftn(C_vals)
    S.real = torch.clamp(S.real, min=0) # 근사 오차로 인한 음수 제거

    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=DEVICE, dtype=DTYPE))
    weighted_freq = torch.sqrt(S.real) * random_phase
    field_sim = torch.fft.ifftn(weighted_freq).real
    
    # 크기가 같으므로 Slicing 없이 그대로 반환
    return field_sim

Simulating on: cpu


In [3]:
# --- 5. EXECUTION (5/4 Expansion Strategy & Cropping) ---

# [설정] 목표 타겟 범위 (Target Domain - Clean Zone)
target_lat_start, target_lat_end = 2.0, -3.0    # Span 5.0 (북 -> 남)
target_lon_start, target_lon_end = 121.0, 131.0 # Span 10.0 (서 -> 동)

# [전략] 5/4배 (1.25배) 확장 (Expansion)
lat_span = abs(target_lat_start - target_lat_end)
lon_span = abs(target_lon_start - target_lon_end)
expansion_factor = 1.25 

lat_padding = (lat_span * expansion_factor - lat_span) / 2  # 0.625
lon_padding = (lon_span * expansion_factor - lon_span) / 2  # 1.25

print(f"Target Grid: Lat {target_lat_start}~{target_lat_end}, Lon {target_lon_start}~{target_lon_end}")
print(f"Padding Added: Lat +/- {lat_padding:.3f}, Lon +/- {lon_padding:.3f}")

# 1. 확장된 시뮬레이션 격자 생성
lats_sim_extended = torch.arange(
    target_lat_start + lat_padding,       # 2.625
    target_lat_end - lat_padding - 0.001, # -3.625
    -0.044, device=DEVICE, dtype=DTYPE
)
lons_sim_extended = torch.arange(
    target_lon_start - lon_padding,       # 119.75
    target_lon_end + lon_padding + 0.001, # 132.25
    0.063, device=DEVICE, dtype=DTYPE
)

t_def = 8
LOC_ERR_STD = 0.01 

# --- 5. EXECUTION (5/4 Expansion Strategy & Cropping) ---

# ... (타겟 설정 및 확장 격자 생성 코드는 그대로 유지) ...

print("1. Generating True Field (Folded)...")

# [수정] generate_exact_gems_field -> generate_folded_gems_field 로 변경
sim_field = generate_folded_gems_field(lats_sim_extended, lons_sim_extended, t_def, params_list)

# ... (이후 Perturbation 및 Cropping 코드는 그대로 유지) ...
# 2. Perturbation & Formatting (Extended Data)
raw_extended_map = {} 
nugget_std = torch.sqrt(torch.exp(params_list[6]))

grid_lat, grid_lon = torch.meshgrid(lats_sim_extended, lons_sim_extended, indexing='ij')
flat_lats = grid_lat.flatten()
flat_lons = grid_lon.flatten()

for t in range(t_def):
    field_t = sim_field[:, :, t]
    flat_vals = field_t.flatten()
    
    # Noise & Perturbation
    obs_vals = flat_vals + (torch.randn_like(flat_vals) * nugget_std) + OZONE_MEAN
    lat_noise = torch.randn_like(flat_lats) * LOC_ERR_STD
    lon_noise = torch.randn_like(flat_lons) * LOC_ERR_STD
    
    # 확장된 데이터 텐서 생성 (Padding 포함)
    row_tensor = torch.stack([
        flat_lats + lat_noise,  # Lat (Perturbed)
        flat_lons + lon_noise,  # Lon (Perturbed)
        obs_vals,               # Val
        torch.full_like(flat_lats, 21.0 + t) # Time
    ], dim=1)
    
    key_str = f'2024_07_y24m07day01_hm{t:02d}:53'
    raw_extended_map[key_str] = row_tensor.detach()

# --- 6. CROPPING & CONNECTING TO DOWNSTREAM ---
print("\n--- Enforcing Regular Grid & Cropping ---")

# 1. 타겟 격자 생성 (Clean Zone Only)
# 패딩 없이 '원래 분석하려던 범위'만 생성
step_lat, step_lon = 0.044, 0.063
target_grid, Nx_reg, Ny_reg = make_target_grid(
    lat_start=target_lat_start, lat_end=target_lat_end, lat_step=-step_lat, 
    lon_start=target_lon_start, lon_end=target_lon_end, lon_step=step_lon,  
    device=DEVICE, dtype=DTYPE
)

# 2. Map & Crop (가장자리 버리기)
# raw_extended_map(확장 데이터)에서 target_grid(중심부)에 맞는 것만 가져옴
# [중요] 변수명 inputmap 유지
inputmap = coarse_by_center_tensor(raw_extended_map, target_grid)

# [중요] aggregated_data 재생성 (Clean Data 기준)
# 이전에 raw_extended_map으로 만든 aggregated_data는 버리고,
# 잘라낸 inputmap을 기준으로 다시 합쳐야 합니다.
aggregated_list = list(inputmap.values())
aggregated_data = torch.cat(aggregated_list, dim=0)

print(f"Final 'inputmap' keys: {len(inputmap)}")
print(f"Final 'aggregated_data' Shape: {aggregated_data.shape}")

# 검증
check_tensor = list(inputmap.values())[0]
print(f"  -> Should match Target Grid Size: {target_grid.shape[0]}")

Target Grid: Lat 2.0~-3.0, Lon 121.0~131.0
Padding Added: Lat +/- 0.625, Lon +/- 1.250
1. Generating True Field (Folded)...
Folded Simulation Grid: 143 x 199 x 8 (No 2x Padding)

--- Enforcing Regular Grid & Cropping ---
Grid Generation debug: Lat Range 2.0000--2.9720, Lon Range 121.0000-130.9540
Unique Lats: 114, Unique Lons: 159
Final 'inputmap' keys: 8
Final 'aggregated_data' Shape: torch.Size([145008, 4])
  -> Should match Target Grid Size: 18126


set up

In [4]:
from GEMS_TCO import orderings as _orderings
import torch
import numpy as np
from typing import Tuple

# inputmap, aggregated_data 변수는 외부에 있다고 가정

def get_spatial_ordering(
        input_maps: dict,
        mm_cond_number: int = 10
    ) -> Tuple[np.ndarray, list]: # 반환 타입힌트 변경 (list)
        
        key_list = list(input_maps.keys())
        data_for_coord = input_maps[key_list[0]]
        
        # Tensor -> Numpy 변환
        if isinstance(data_for_coord, torch.Tensor):
            data_for_coord = data_for_coord.cpu().numpy()

        x1 = data_for_coord[:, 0]
        y1 = data_for_coord[:, 1]
        
        coords1 = np.stack((x1, y1), axis=-1)

        # 1. MaxMin Ordering
        ord_mm = _orderings.maxmin_cpp(coords1)
        
        # 2. Reorder coordinates
        data_for_coord_reordered = data_for_coord[ord_mm]
        coords1_reordered = np.stack(
            (data_for_coord_reordered[:, 0], data_for_coord_reordered[:, 1]), 
            axis=-1
        )
        
        # 3. Calculate nearest neighbors map (Dictionary 반환됨)
        nns_map_dict = _orderings.find_nns_l2(locs=coords1_reordered, max_nn=mm_cond_number)
        
        # --- 🔴 [FIX 1] Dictionary -> List 변환 (TypeError 방지) ---
        # 키(Key) 순서대로 값(Value)만 뽑아서 리스트로 만듭니다.
        nns_map_list = [nns_map_dict[i] for i in range(len(nns_map_dict))]
        # -----------------------------------------------------
        
        return ord_mm, nns_map_list

# --- 🔴 [FIX 2] mm_cond_number 변경 (16 -> 10) ---
# 16개로 하면 메모리(35)가 터집니다. 3*10 + 5 = 35 이므로 10이 한계입니다.
ord_mm, nns_map = get_spatial_ordering(inputmap, mm_cond_number=15)

# 데이터 재정렬 (이건 그대로 사용)
mm_input_map = {}
for key in inputmap:
    mm_input_map[key] = inputmap[key][ord_mm]

Likelihood vs. Truth: A model with the wrong parameters (short range) might produce a higher Vecchia likelihood because it fits the high-frequency noise better, but it will be terrible at prediction (Kriging) away from data points.

# Fit vecchia max min time 2 

In [11]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 8   # Neighbors
#mm_cond_number = 16   # Neighbors
nheads = 300           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 3
LBFGS_HISTORY_SIZE = 100 # 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 30    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

# [4.2042, 1.6348, 0.4721, -3.2695, 0.0218, -0.1689, -1.3984]
params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(aggregated_data, torch.Tensor):
    aggregated_data = aggregated_data.to(DEVICE)

# Instantiate
model_instance = kernels_reparam_space_time_gpu.fit_vecchia_lbfgs(
    smooth=v,
    input_map=mm_input_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads=nheads
)

# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
🚀 Pre-computing (Corrected Vectorization)... ✅ Done in 1.0000s. (Heads: 2400, Tails: 142608)
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/3 / Loss: 1.249389 ---
  Param 0: Value=4.2541, Grad=-1.5762379574032798e-07
  Param 1: Value=1.7243, Grad=9.95809473811304e-08
  Param 2: Value=0.3339, Grad=-3.099168185039211e-07
  Param 3: Value=-2.3352, Grad=1.150912478730759e-07
  Param 4: Value=0.0621, Grad=-2.313199227550137e-07
  Param 5: Value=-0.1779, Grad=-5.3748181868922e-07
  Param 6: Value=-1.4740, Grad=-1.919515572459652e-07
  Max Abs Grad: 5.374818e-07
------------------------------
--- Step 2/3 / Loss: 1.247604 ---
  Param 0: Value=4.2541, Grad=-1.5762379574032798e-07
  Param 1: Value=1.7243, Grad=9.95809473811304e-08
  Param 2: Value=0.3339, Grad=-3.099168185039211e-07
  Param 3: Value=-2.3352, Grad=1.150912478730759e-07
  Param 4: Value=0.0621, Grad=-2.313199227550137e

In [ ]:
Final Interpretable Params: {'sigma_sq': 12.550968451994, 
                             'range_lon': 0.17829175252165666,
                               'range_lat': 0.15087767760409757, 
                               'range_time': 0.5730659105565772,
                                 'advec_lat': 0.06208971176953433,
                                   'advec_lon': -0.17790341869704374,
                                     'nugget': 0.22901237722387074}

Optimization finished in 118.26s.
Results after 9 steps: [4.254131841461119, 1.7243340113710215, 0.33390368236413404, -2.335158939275716, 0.06208971176953433, -0.17790341869704374, -1.4739792278759567, 1.2476043286412608]
Final Params: [ 4.25413184  1.72433401  0.33390368 -2.33515894  0.06208971 -0.17790342
 -1.47397923]

# fit dw

difference data

In [25]:
a = [11.0474, 0.0623, 0.2445, 1.0972, 0.0101, -0.1671, 1.1825]
day = 0 # 0 index
lat_range= [0,5]
lon_range= [123.0, 133.0]
#lat_range= [1,3]
#lon_range= [125, 129.0]

daily_aggregated_tensors_dw = [aggregated_data]
daily_hourly_maps_dw = [input_map]

db = debiased_whittle.debiased_whittle_preprocess(daily_aggregated_tensors_dw, daily_hourly_maps_dw, day_idx=day, params_list=a, lat_range=lat_range, lon_range=lon_range)


subsetted_aggregated_day = db.generate_spatially_filtered_days(0,5,123,133)
print(subsetted_aggregated_day.shape)
N2= subsetted_aggregated_day.shape[0]
print(N2)
subsetted_aggregated_day[:20]

torch.Size([142832, 4])
142832


tensor([[ 2.8000e-02,  1.2305e+02,  0.0000e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2311e+02,  1.9113e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2317e+02,  3.1846e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2323e+02,  9.3044e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2330e+02,  7.5384e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2336e+02, -8.2040e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2342e+02, -1.2852e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2349e+02, -3.2619e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2355e+02, -6.2629e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2361e+02,  1.3124e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2368e+02,  4.2001e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2374e+02, -4.3216e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2380e+02, -5.5636e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2386e+02, -2.0360e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2393e+02,  4.5638e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2399e+02, -5.2148e-02,  2.1000e+01],
        

In [26]:

dwl = debiased_whittle.debiased_whittle_likelihood()
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 3 # data is decided above
    TAPERING_FUNC = dwl.cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    MAX_STEPS = 20 # L-BFGS usually converges in far fewer steps
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3


    cur_df =subsetted_aggregated_day
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} is empty or invalid.")
        exit()

    unique_times = torch.unique(cur_df[:, TIME_COL])
    time_slices_list = [cur_df[cur_df[:, TIME_COL] == t_val] for t_val in unique_times]

    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    
    # --- 💥 REVISED: Renamed 'p' to 'p_time' 💥 ---
    J_vec, n1, n2, p_time, taper_grid = dwl.generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p_time == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = dwl.calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = dwl.calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p_time} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")
    # --- END REVISION ---

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # --- 7-PARAMETER initialization ---
        ''' 
        init_sigmasq   = 15.0
        init_range_lat = 0.66 
        init_range_lon = 0.7 
        init_nugget    = 1.5
        init_beta      = 0.1  # Temporal range ratio
        init_advec_lat = 0.02
        init_advec_lon = -0.08
        '''
        init_sigmasq   = 13.059
        init_range_lat = 0.154 
        init_range_lon = 0.195
        init_advec_lat = 0.0218
        init_range_time = 0.7
        init_advec_lon = -0.1689
        init_nugget    = 0.247

        init_phi2 = 1.0 / init_range_lon
        init_phi1 = init_sigmasq * init_phi2
        init_phi3 = (init_range_lon / init_range_lat)**2
        # Change needed to match the spatial-temporal distance formula:
        init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

        initial_params_values = [
            np.log(init_phi1),    # [0] log_phi1
            np.log(init_phi2),    # [1] log_phi2
            np.log(init_phi3),    # [2] log_phi3
            np.log(init_phi4),    # [3] log_phi4
            init_advec_lat,       # [4] advec_lat (NOT log)
            init_advec_lon,       # [5] advec_lon (NOT log)
            np.log(init_nugget)   # [6] log_nugget
        ]
        
        print(f"Starting with FIXED params (raw log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float64))
            for val in initial_params_values
        ]

        # Helper to define the boundary globally for clarity
        NUGGET_LOWER_BOUND = 0.05
        LOG_NUGGET_LOWER_BOUND = np.log(NUGGET_LOWER_BOUND) # Approx -2.9957

        # --- 💥 REVISED: Use L-BFGS Optimizer 💥 ---
        optimizer = torch.optim.LBFGS(
            params_list,
            lr=1.0,           # Initial step length for line search
            max_iter=20,      # Iterations per step
            history_size=100,
            line_search_fn="strong_wolfe", # Often more robust
            tolerance_grad=1e-5
        )
        # --- END REVISION ---

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, 7-param ST kernel, L-BFGS)...")
        
        # --- 💥 REVISED: Call L-BFGS trainer, pass p_time 💥 ---
        nat_params_str, phi_params_str, raw_params_str, loss, steps_run = dwl.run_lbfgs_tapered(
            params_list=params_list,
            optimizer=optimizer,
            I_sample=I_sample,
            n1=n1, n2=n2, p_time=p_time,
            taper_autocorr_grid=taper_autocorr_grid, 
            max_steps=MAX_STEPS,
            device=DEVICE
        )
        # --- END REVISION ---
        
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str, raw_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\n{'='*25} Overall Result from Run {'='*25} {'='*25}")
    
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        print(f"Best Run Loss: {best_loss} (after {steps_run} steps)")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")
        print(f"Final Parameters (Raw Log Scale): {best_results[2]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 113x158, 8 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (raw log-scale): [4.2042, 1.6348, 0.4721, -2.5562, 0.0218, -0.1689, -1.3984]
Starting optimization run 1 on device cpu (Hamming, 7-param ST kernel, L-BFGS)...
--- Step 1/20 ---
 Loss: 1.906929 | Max Grad: 6.274846e-04
  Params (Raw Log): log_phi1: 4.3182, log_phi2: 1.7745, log_phi3: 0.0776, log_phi4: -3.5323, advec_lat: 0.0363, advec_lon: -0.1510, log_nugget: -1.0186
--- Step 2/20 ---
 Loss: 1.817385 | Max Grad: 4.095367e-05
  Params (Raw Log): log_phi1: 4.3182, log_phi2: 1.7744, log_phi3: 0.0775, log_phi4: -3.5320, advec_lat: 0.0363, advec_lon: -0.1510, log_nugget: -1.0177
--- Step 3/20 ---
 Loss: 1.817385 | Max Grad: 4.095367e-05
  Params (Raw Log): log

init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

1 st simulation (1 vs 3)

Final Interpretable Params: {'sigma_sq': 12.939260896610579, 'range_lon': 0.17242543281507933, 'range_lat': 0.1631231346200311, 'range_time': 1.1358209227858689, 'advec_lat': 0.045119029109988974, 'advec_lon': -0.17809677784942035, 'nugget': 0.3009582276680578}

Final Parameters (Natural Scale): sigmasq: 13.2415, range_lat: 0.1685, range_lon: 0.1739, range_time: 0.9681, advec_lat: 0.0395, advec_lon: -0.1732, nugget: 0.3216

2nd simulation (1 vs 3)

Final Interpretable Params: {'sigma_sq': 12.765293287952144, 'range_lon': 0.17039368470743024, 'range_lat': 0.16152132799710625, 'range_time': 1.081124889959751, 'advec_lat': 0.04635511983762563, 'advec_lon': -0.1775715292039452, 'nugget': 0.31503884896742074}


sigmasq: 13.2415, range_lat: 0.1685, range_lon: 0.1739, range_time: 0.9681, advec_lat: 0.0395, advec_lon: -0.1732, nugget: 0.3216

3nd simulation (mm 15 two times)

Final Interpretable Params: {'sigma_sq': 12.292570063933866, 'range_lon': 0.16170186578939172, 'range_lat': 0.1537053192245325, 'range_time': 0.9671689003322674, 'advec_lat': 0.04155171168950867, 'advec_lon': -0.16218073366456207, 'nugget': 0.29945872751676345}

Final Parameters (Natural Scale): sigmasq: 12.7277, range_lat: 0.1631, range_lon: 0.1696, range_time: 0.9917, advec_lat: 0.0363, advec_lon: -0.1510, nugget: 0.3614
